In [67]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

from PhaseFunctionedNetwork import PhaseFunctionedNetwork
from train_utils import train_pfnn

# set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)
rng = np.random.RandomState(42)

# removed third style to see if that is what is making motion data stiff (IT WAS)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
# Process Data 
ROOT_PATH = 'C:/Users/Ana/Desktop/dev/pfnn-dev/Export/'
INPUT_PATH = 'Input_'
OUTPUT_PATH = 'Output_'

def get_change_in_phase(phase_arr):
    change_in_phase =  phase_arr[1:] - phase_arr[:-1]
    change_in_phase[change_in_phase < 0] = (1.0 - phase_arr[:-1] + phase_arr[1:])[change_in_phase < 0]
    change_in_phase = np.append(change_in_phase,  change_in_phase[-1]) #TODO IF BREAKS append 2pi
    return change_in_phase

def load_data(action_name, num_decimals):
    X_arr = np.float32(np.loadtxt(ROOT_PATH + INPUT_PATH + action_name))
    Y_arr = np.float32(np.loadtxt(ROOT_PATH + OUTPUT_PATH + action_name))
    P_arr = np.round(X_arr[:, -1], num_decimals)
    # remove Phase from X so it is not modified during preprocessing
    X_arr = X_arr[:, :-1]
    
    return X_arr, Y_arr, P_arr

In [69]:
# Load Data
round = 5
# Idle Data
X_idle, Y_idle, P_idle = load_data('Idle_Static.txt', round)
delta_idle = get_change_in_phase(P_idle)
print(f"Idle data shape: {X_idle.shape}, {Y_idle.shape}, {P_idle.shape}, {delta_idle.shape}")

# Jump HorizFlat
X_jump_horizflat, Y_jump_horizflat, P_jump_horizflat = load_data('Jump_HorizFlat.txt', round)
delta_jump_horizflat = get_change_in_phase(P_jump_horizflat)
print(f"Horizflat data shape: {X_jump_horizflat.shape}, {Y_jump_horizflat.shape}, {P_jump_horizflat.shape}, {delta_jump_horizflat.shape}")

# Jump HorizUp
X_jump_horizup, Y_jump_horizup, P_jump_horizup = load_data('Jump_HorizUp.txt', round)
delta_jump_horizup = get_change_in_phase(P_jump_horizup)
print(f"Horizup data shape: {X_jump_horizup.shape}, {Y_jump_horizup.shape}, {P_jump_horizup.shape}, {delta_jump_horizup.shape}")


Idle data shape: (72, 960), (72, 891), (72,), (72,)
Horizflat data shape: (50, 960), (50, 891), (50,), (50,)
Horizup data shape: (48, 960), (48, 891), (48,), (48,)


In [70]:
# WEIGH THE JOINT (in particular all the hand and foot joints need to be weighted less)
JOINT_NUM = 144

joint_weights = np.array([
    1,                                                  # HIPS
    1, 1, 1,                                            # LEFT leg
    1e-10, 1e-10, 1e-10, 1,                             # LEFT foot thumb
    1,                                                  # LEFT foot toe base
    1e-10, 1e-10, 1e-10, 1,                             # LEFT foot index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT foot middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT foot ring
    1, 1, 1,                                            # RIGHT leg
    1e-10, 1e-10, 1e-10, 1,                             # RIGHT foot thumb
    1,                                                  # RIGHT foot toe base
    1e-10, 1e-10, 1e-10, 1,                             # RIGHT foot index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT foot middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT foot ring
    1, 1, 1, 1,                                         # SPINE
    1, 1,                                               # LEFT shoulder
    1, 1,                                               # LEFT arm
    1,                                                  # LEFT hand
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand ring
    1, 1e-10, 1,                                        # LEFT wing feathers large
    1, 1e-10, 1,                                        # LEFT wing feathers medium
    1,                                                  # LEFT wing feathers small
    1, 1, 1, 1, 1, 1, 1,                                # NECK
    1,                                                  # HEAD
    1e-10, 1e-10,                                       # JAW
    1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10,    # TONUGE
    1e-10, 1e-10, 1e-10,                                # LEFT eye
    1e-10, 1e-10, 1e-10,                                # RIGHT eye
    1, 1,                                               # RIGHT shoulder
    1, 1,                                               # RIGHT arm
    1,                                                  # RIGHT hand
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand ring
    1, 1e-10, 1,                                        # RIGHT wing feathers large
    1, 1e-10, 1,                                        # RIGHT wing feathers medium
    1,                                                  # RIGHT wing feathers small
    1, 1, 1, 1, 1, 1, 1, 1, 1,                          # TAIL
    1e-10, 1e-10, 1,                                    # LEFT tail feather
    1e-10, 1e-10, 1,                                    # MIDDLE tail feather         
    1e-10, 1e-10, 1                                     # RIGHT tail feather 
])

# repeat weights for each joint to represent X, Y and Z
joint_weights = joint_weights.repeat(3)

In [71]:
def preprocess_X(X_arr):
    Xmean, Xstd = X_arr.mean(axis=0), X_arr.std(axis=0)

    # lists to keep track of indices for TRAJECTORY
    X_traj_pos_indices = []
    X_traj_dir_indices = []
    X_traj_slope_indices = []
    X_traj_style_indices = []

    # number of eleements for each trajectory point
    w = 8
    for i in range(0, 95, w):
        X_traj_pos_indices = np.append(X_traj_pos_indices, range(i,i+3)).astype(int)
        X_traj_dir_indices = np.append(X_traj_dir_indices, range(i+3,i+5)).astype(int)
        X_traj_slope_indices = np.append(X_traj_slope_indices, i+5).astype(int)
        X_traj_style_indices = np.append(X_traj_style_indices, range(i+6,i+8)).astype(int)

    # lists to keep track of indices for JOINTS
    X_joint_pos_indices = []
    X_joint_vel_indices = []

    # num of elements for each joint
    w = 6
    for i in range(96, 959, w):
        X_joint_pos_indices = np.append(X_joint_pos_indices, range(i,i+3)).astype(int)
        X_joint_vel_indices = np.append(X_joint_vel_indices, range(i+3,i+6)).astype(int)

    # INPUT Trajectory data
    Xstd[X_traj_pos_indices] = Xstd[X_traj_pos_indices].mean()
    Xstd[X_traj_dir_indices] = Xstd[X_traj_dir_indices].mean()
    Xstd[X_traj_style_indices] = Xstd[X_traj_style_indices].mean()
    Xstd[X_traj_slope_indices] = Xstd[X_traj_slope_indices].mean()

    # INPUT Joint data --> This is where we weight the joints
    # Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() / (joint_weights * 0.1)
    # Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean() / (joint_weights * 0.1)
    Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() 
    Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean()

    return Xmean, Xstd

def preprocess_Y(Y_arr):
    Ymean, Ystd = Y_arr.mean(axis=0), Y_arr.std(axis=0)

    # PREPROCESS OUTPUT Y
    # lists to keep track of indices for TRAJECTORY
    Y_traj_pos_indices = []
    Y_traj_dir_indices = []

    # number of trajectory elements
    w = 4
    for i in range(0, 23, w): #TODO UPDATE THE RANGE
        Y_traj_pos_indices = np.append(Y_traj_pos_indices, range(i,i+2)).astype(int)
        Y_traj_dir_indices = np.append(Y_traj_dir_indices, range(i+2,i+4)).astype(int)

    # lists to keep track of indices for JOINTS
    Y_joint_pos_indices = []
    Y_joint_vel_indices = []

    # num of joint elements
    w = 6
    for i in range(24, 887, w): #TODO UPDATE THE RANGE
        Y_joint_pos_indices = np.append(Y_joint_pos_indices, range(i,i+3)).astype(int)
        Y_joint_vel_indices = np.append(Y_joint_vel_indices, range(i+3,i+6)).astype(int)

    # OUTPUT Trajectory data
    Ystd[Y_traj_pos_indices] = Ystd[Y_traj_pos_indices].mean()
    Ystd[Y_traj_dir_indices] = Ystd[Y_traj_dir_indices].mean()

    # OUTPUT Joint data --> This is where we weight the joints
    # Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean() / (joint_weights * 0.1)
    # Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean() / (joint_weights * 0.1)
    Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean()
    Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean()
    
    # translational_vel_mean = (Ystd[-4] + Ystd[-2])/2
    # Ystd[-4] = translational_vel_mean
    # Ystd[-2] = translational_vel_mean

    return Ymean, Ystd

In [77]:
P_jump_horizup

array([0.63222, 0.62444, 0.61667, 0.60889, 0.60111, 0.59333, 0.58556,
       0.57778, 0.57   , 0.56222, 0.55444, 0.54667, 0.53889, 0.53111,
       0.52333, 0.51556, 0.50778, 0.     , 0.50452, 0.50903, 0.51355,
       0.51806, 0.52258, 0.5271 , 0.53161, 0.53613, 0.54065, 0.54516,
       0.54968, 0.55419, 0.55871, 0.56323, 0.56774, 0.57226, 0.57677,
       0.58129, 0.58581, 0.59032, 0.59484, 0.59935, 0.60387, 0.60839,
       0.6129 , 0.61742, 0.62194, 0.62645, 0.63097, 0.63548],
      dtype=float32)

In [76]:
delta_jump_horizup

array([0.99222004, 0.99223   , 0.99222   , 0.99222   , 0.99222004,
       0.99223   , 0.99222   , 0.99222   , 0.99222   , 0.99222004,
       0.99223   , 0.99222   , 0.99222   , 0.99222   , 0.99223   ,
       0.99222004, 0.49221998, 0.50452   , 0.00450999, 0.00452   ,
       0.00451005, 0.00452   , 0.00452   , 0.00450999, 0.00452   ,
       0.00452   , 0.00450999, 0.00452   , 0.00450999, 0.00452   ,
       0.00452   , 0.00451005, 0.00452   , 0.00450999, 0.00452   ,
       0.00452   , 0.00450999, 0.00452   , 0.00450999, 0.00452   ,
       0.00452   , 0.00451005, 0.00452   , 0.00452   , 0.00450999,
       0.00452   , 0.00450999, 0.00450999], dtype=float32)

In [72]:
# Concatenate Data into indivudal X, Y, P Arrays
X = np.concatenate((X_idle, X_jump_horizflat, X_jump_horizup))
Y = np.concatenate((Y_idle, Y_jump_horizflat, Y_jump_horizup))
P = np.concatenate((P_idle, P_jump_horizflat, P_jump_horizup))
delta_phase = np.concatenate((delta_idle, delta_jump_horizflat, delta_jump_horizup))
Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

# Preprocess Data
Xmean, Xstd = preprocess_X(X)
Ymean, Ystd = preprocess_Y(Y)

WEIGHTS_SAVE_PATH = 'C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/'

for i in range(Xstd.size):
    if (Xstd[i]==0):
        Xstd[i]=1
for i in range(Ystd.size):
    if (Ystd[i]==0):
        Ystd[i]=1

# save means and stds
Xmean.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Xmean.bin')
Ymean.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Ymean.bin')
Xstd.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Xstd.bin')
Ystd.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Ystd.bin')

# normalize data NOTE ORIGINAL DID THIS AFTER SAVING THE MEANS AND STD AS DONE HERE
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

# load data for PyTorch training

# append phase as additional feature only for training NN
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

print(f"Input shape {input.shape}")
print(f"Target shape {target.shape}")

dataset = TensorDataset(input, target)

Input shape torch.Size([170, 961])
Target shape torch.Size([170, 892])


In [75]:
# Define PFNN
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=target.shape[1], dropout=0.9)

# Determine device for training 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)

# Training variables
BATCH_SIZE = 32
EPOCHS = 5
LR = 0.001
OPTIMIZER = torch.optim.AdamW(model.parameters(), lr=LR)
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Train
model = train_pfnn(model, train_dataloader, optimizer=OPTIMIZER, num_epochs=EPOCHS, device=DEVICE)

# Save
model.precompute_and_save_weights()

100%|██████████| 6/6 [00:00<00:00,  9.29it/s]


Epoch [1/5], Loss: 1.397931456565857


100%|██████████| 6/6 [00:00<00:00, 15.59it/s]


Epoch [2/5], Loss: 1.5598622957865398


100%|██████████| 6/6 [00:00<00:00, 15.61it/s]


Epoch [3/5], Loss: 1.231041411558787


100%|██████████| 6/6 [00:00<00:00, 15.63it/s]


Epoch [4/5], Loss: 0.9666406015555064


100%|██████████| 6/6 [00:00<00:00, 15.63it/s]


Epoch [5/5], Loss: 0.8187666932741801
